In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from collections import defaultdict
import os
import re

In [8]:
authors_short_dict = {"MWS": "Mary Shelley", "HPL": "H. P. Lovecraft", "EAP": "Edgar Allan Poe"}
df = pd.DataFrame(pd.read_json("preprocessing_output/preprocessed_test_W.json").author)
df["author_short"] = df.author
df.author = df.author.apply(lambda x: authors_short_dict[x])
for filename in os.listdir("results_output/"):
    fileparts = filename.split("_")
    method = fileparts[1]
    if "template" in method:
        continue
    preprocessing = ".".join(fileparts[2].split(".")[:-1])
    df[f"{method} with {preprocessing}"] = pd.read_json(f"results_output/results_{method}_{preprocessing}.json")
# df

metrics = [("Accuracy", accuracy_score), ("Precision", lambda x, y: precision_score(x, y, average="macro")), ("Recall",
                                                                                                              lambda x, y: recall_score(x, y, average="macro")), ("F1 Score", lambda x, y: f1_score(x, y, average="macro"))]

In [9]:
total_table = pd.DataFrame()
for (name, func) in metrics:
    for method in ["bayes", "lda"]:
        authors = defaultdict(lambda: [])
        for author in ["EAP", "HPL", "MWS"]:
            method_df = df.loc[:, df.columns.str.contains(f"(?=.*author)|(?=.*{method})")]
            author_df = pd.DataFrame({"author": method_df["author"]})
            author_df = pd.concat([author_df, method_df.iloc[:, 1:].apply(
                    lambda x: [1 if author == i else 0 for i in x])], axis=1) 
            author_df = author_df.rename(columns=lambda x: re.sub(r".*\s(\S+)$", r"\1", x))
            authors[author] = author_df.iloc[:, 2:].apply(lambda x: func(author_df["author_short"], x))
        authors = pd.DataFrame(authors)
        total_table = pd.concat([total_table, authors], axis=1)
pd.set_option('display.max_columns', None)
total_table.columns = pd.MultiIndex.from_product([map(lambda x: x[0], metrics), ["bayes", "lda"], ["EAP", "HPL", "MWS"]])
total_table

NameError: name 'defaultdict' is not defined

In [ ]:
for method in ["bayes", "lda"]:
    method_df = df.loc[:, df.columns.str.contains(f"^(?:(?=.*\\b(?:author|{method}|author_short)\\b)(?:(?![56]).)*)$")]
    method_df = method_df.rename(columns=lambda x: re.sub(r".*\s(\S+)$", r"\1", x))
    score_df = method_df.iloc[: , 2:].apply(lambda x: [accuracy_score(x, method_df["author_short"]), precision_score(method_df["author_short"], x, average="macro"), recall_score(method_df["author_short"], x, average="macro"), f1_score(method_df["author_short"], x, average="macro")])
    score_df["metric"] = ["accuracy", "precision", "recall", "F1 score"]
    score_df.plot.bar("metric", title=f"Overview {method}")
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

In [ ]:
for method in ["bayes", "lda"]:
    for (name, func) in metrics:
        score_df = {}
        method_df = df.loc[:, df.columns.str.contains(
            f"^(?:(?=.*\\b(?:author|{method}|author_short)\\b)(?:(?![56]).)*)$")]
        for author in ["EAP", "MWS", "HPL"]:
            author_df = pd.DataFrame(
                {"author": method_df["author"]})
            author_df = pd.concat([author_df, method_df.iloc[:, 1:].apply(
                lambda x: [1 if author == i else 0 for i in x])], axis=1)
            author_df = author_df.rename(columns=lambda x: re.sub(r".*\s(\S+)$", r"\1", x))
            score_df[author] = author_df.iloc[:, 2:].apply(
                lambda x: func(author_df["author_short"], x))
        score_df = pd.DataFrame(score_df)
        score_df.plot.bar(title=f"Author Overview {method} {name}")
        plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')


In [ ]:
for (name, func) in metrics:
    preprocess_df = {}
    for method in ["bayes", "lda"]:
        score_df = df.loc[:, df.columns.str.contains(f"^(?:(?=.*\\b(?:author|{method}|author_short)\\b)(?:(?![56]).)*)$")]
        score_df = score_df.rename(columns=lambda x: re.sub(r".*\s(\S+)$", r"\1", x))
        score_df = score_df.iloc[:, 2:].apply(lambda x: func(score_df["author_short"], x))
        preprocess_df[method] = score_df
    preprocess_df = pd.DataFrame(preprocess_df)
    preprocess_df.plot.bar(title=f"Method Comparison Overview {name}")
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

In [ ]:
for (name, func) in metrics:
    method_df = df.loc[:, df.columns.str.contains(f"^[^56]*$")]
    for preprocessing in range(2,int((len(method_df.columns)) / 2 + 1)):
        authors = defaultdict(lambda: [])
        preprocess_name = ""
        for author in ["EAP", "HPL", "MWS"]:
            author_df = pd.DataFrame(
                            {"author": method_df["author"]})
            author_df = pd.concat([author_df, method_df.iloc[:, 1:].apply(
                lambda x: [1 if author == i else 0 for i in x])], axis=1)
            for method in ["bayes", "lda"]:
                process_df = author_df.loc[:, author_df.columns.str.contains(f"(.*{method}.*)|(.*author.*)", regex=True)]
                preprocess_name = re.sub(".*\s(\S+)$", r"\1", process_df.columns[preprocessing])
                authors[author].append(func(process_df["author_short"],process_df.iloc[:, preprocessing]))
        authors = pd.DataFrame(authors)
        authors = authors.rename(columns=lambda x: authors_short_dict[x])
        authors["method"] = ["bayes", "lda"]
        authors = authors.pivot_table(columns ="method")
        authors.plot.bar(title=f"Author Comparison Overview {name} with {preprocess_name}")
        plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

In [ ]:
for method in ["bayes", "lda"]:
    method_df = df.loc[:, df.columns.str.contains(f"(author|{method})")]
    for preprocessing in range(2,int((len(method_df.columns)))):
        preprocess_name = re.sub(r".*\s(\S+)$", r"\1", method_df.columns[preprocessing])
        score_df = {}
        for author in ["EAP", "MWS", "HPL"]:
            author_df = pd.DataFrame(
                {"author": method_df["author"]})
            author_df = pd.concat([author_df, method_df.iloc[:, 1:].apply(
                lambda x: [1 if author == i else 0 for i in x])], axis=1)
            author_df = author_df.rename(columns=lambda x: re.sub(r".*\s(\S+)$", r"\1", x))
            score_df[author] = [func(author_df["author_short"],author_df.iloc[:, preprocessing]) for (name, func) in metrics]
        score_df = pd.DataFrame(score_df)
        score_df.index = ["accuracy", "precision", "recall", "F1 score"]
        score_df.plot.bar(title=f"Authors with {preprocess_name} using {method}")
        plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
        

In [ ]:
for method in ["bayes", "lda"]:
    method_df = df.loc[:, df.columns.str.contains(f"(?=.*author)|(?=.*{method})")]
    for preprocessing in range(2,int((len(method_df.columns)))):
        preprocess_name = re.sub(r".*\s(\S+)$", r"\1", method_df.columns[preprocessing])
        display(method_df.groupby(["author_short", method_df.columns[preprocessing]]).size().unstack(fill_value=0))

In [ ]:
for method in ["bayes", "lda"]:
    for preprocessing in ["WSPH", "CSPH"]:
        method_df = df.loc[:, df.columns.str.contains(f"(?=.*author)|(?=.*{method})(?=.*{preprocessing})")]
        method_df = method_df.rename(columns=lambda x: re.sub(r".*\s(\S+)$", r"\1", x))
        score_df = method_df.iloc[: , 2:].apply(lambda x: [accuracy_score(x, method_df["author_short"]), precision_score(method_df["author_short"], x, average="macro"), recall_score(method_df["author_short"], x, average="macro"), f1_score(method_df["author_short"], x, average="macro")])
        score_df.index = ["accuracy", "precision", "recall", "F1 score"]
        score_df = score_df.rename(columns=lambda x: re.sub(r'^\D*(\d.*)', r'\1', x))
        score_df.plot.bar(title=f"Highpass Comparison with {preprocessing} using {method}")
        plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

In [ ]:
for (name, func) in metrics:
    for method in ["bayes", "lda"]:
        for preprocessing in ["WSPH", "CSPH"]:
            method_df = df.loc[:, df.columns.str.contains(f"(?=.*author)|(?=.*{method})(?=.*{preprocessing})")]
            cutoffs = defaultdict(lambda: [])
            for author in ["EAP", "HPL", "MWS"]:
                author_df = pd.DataFrame(
                    {"author": method_df["author"]})
                author_df = pd.concat([author_df, method_df.iloc[:, 1:].apply(
                    lambda x: [1 if author == i else 0 for i in x])], axis=1)
                for cutoff in range(2,int(len(author_df.columns))):
                    cutoffs[author_df.columns[cutoff]].append(func(author_df["author_short"], author_df.iloc[:, cutoff]))
            cutoffs = pd.DataFrame(cutoffs)
            cutoffs.index = [authors_short_dict[author] for author in ["EAP", "HPL", "MWS"]]
            cutoffs = cutoffs.rename(columns=lambda x: re.sub(r'^\D*(\d.*)', r'\1', x))
            cutoffs.plot.bar(title=f"Author Highpass Comparison with {preprocessing} using {method}")
            plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
